In [2]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Run some stuff

In [3]:
import pathlib

import numpy as np
import pandas as pd
import xarray as xr
from model_scripts import gr4j_cema_neige, hbv
from scripts.data_readers import metric_df, model_ready_file
from tqdm.notebook import tqdm

### Get unique composition based on alghoritm and metric

#### GR4J

In [37]:
gr4j_models = list(pathlib.Path("./").glob("gr4j_calibration*"))

for gr4j_mdl in gr4j_models:
    algh, metric = str(gr4j_mdl).split("_")[-2], str(gr4j_mdl).split("_")[-1]

    gauges = [i.stem for i in pathlib.Path(f"./{gr4j_mdl}/").glob("*.npy")]

    gr4j_res_df = list()
    for gauge_id in tqdm(gauges):
        with xr.open_dataset(
            f"../geo_data/ws_related_meteo/nc_all_q/{gauge_id}.nc"
        ) as f:
            example_df = f.to_pandas()
            example_df = example_df.drop("gauge_id", axis=1)
            test_df = model_ready_file(example_df["2019-01-01":])

        with open(f"./{gr4j_mdl}/{gauge_id}.npy", "rb") as f:
            gauge_params = np.load(f)[0]

        res_df = pd.DataFrame()
        res_df["obs"] = test_df["Q_mm"]
        res_df["pred"] = gr4j_cema_neige.simulation(
            test_df, params=gauge_params)
        gr4j_res_df.append(
            metric_df(
                gauge_id=gauge_id, predictions=res_df["pred"], targets=res_df["obs"]
            )
        )
    gr4j_res_df = pd.concat(gr4j_res_df)
    gr4j_res_df.index.name = "gauge_id"
    gr4j_res_df.to_csv(f"./cal_res/gr4j/GR4J_res_{algh}_{metric}.csv")

  0%|          | 0/1112 [00:00<?, ?it/s]

  0%|          | 0/1112 [00:00<?, ?it/s]

  0%|          | 0/1112 [00:00<?, ?it/s]

  0%|          | 0/1112 [00:00<?, ?it/s]

  0%|          | 0/1105 [00:00<?, ?it/s]

  0%|          | 0/1112 [00:00<?, ?it/s]

#### HBV

In [39]:
hbv_models = list(pathlib.Path("./").glob("hbv_calibration*"))

for hbv_mdl in hbv_models:
    algh, metric = str(hbv_mdl).split("_")[-2], str(hbv_mdl).split("_")[-1]

    gauges = [i.stem for i in pathlib.Path(f"./{hbv_mdl}/").glob("*.npy")]

    hbv_res_df = list()
    for gauge_id in tqdm(gauges):
        with xr.open_dataset(
            f"../geo_data/ws_related_meteo/nc_all_q/{gauge_id}.nc"
        ) as f:
            example_df = f.to_pandas()
            example_df = example_df.drop("gauge_id", axis=1)
            test_df = model_ready_file(example_df["2019-01-01":])

        with open(f"./{hbv_mdl}/{gauge_id}.npy", "rb") as f:
            gauge_params = np.load(f)[0]

        res_df = pd.DataFrame()
        res_df["obs"] = test_df["Q_mm"]
        res_df["pred"] = hbv.simulation(test_df, params=gauge_params)
        hbv_res_df.append(
            metric_df(
                gauge_id=gauge_id, predictions=res_df["pred"], targets=res_df["obs"]
            )
        )
    hbv_res_df = pd.concat(hbv_res_df)
    hbv_res_df.index.name = "gauge_id"
    hbv_res_df.to_csv(f"./cal_res/hbv/HBV_res_{algh}_{metric}.csv")

  0%|          | 0/1114 [00:00<?, ?it/s]

  0%|          | 0/1114 [00:00<?, ?it/s]

  0%|          | 0/1114 [00:00<?, ?it/s]

  0%|          | 0/1114 [00:00<?, ?it/s]

  0%|          | 0/1114 [00:00<?, ?it/s]

  0%|          | 0/1114 [00:00<?, ?it/s]